In [1]:
import os
import pandas as pd
import numpy as np
import pprint
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import json


# Services

In [2]:
def nativeLang2LangCode(lang):
    lang = lang.lower()
    json_dir = os.path.join(os.getcwd()+'/json/nativelanguage.json')
    # with open(json_dir,encoding='utf-8') as data_file: 
    new_lang =lang
    with open(json_dir) as data_file: 
        data = json.load(data_file)
        for dic in data:
            if (data[dic]['name'].lower() == lang) or (data[dic]['nativeName'].lower() == lang):
#                 print(data[dic]['name'])
                new_lang =  dic
    return new_lang
nativeLang2LangCode('Hebrew')

'he'

In [3]:
def languageCode2ethnicity(languagecode):
    file_path = os.path.join(os.getcwd() + '/dictionary/language_continent_dict.csv')
#     print(file_path)
    language_df = pd.read_csv(file_path)
    if languagecode == 'es':
        return 'Hispanic/Latino'
    else:
        continentcode_df = language_df.loc[language_df.name == languagecode]
        if continentcode_df.empty:
            continentcode = 'UNKNOWN'
        else:
            continentcode = continentcode_df['dorminant_continent'].values[0]
#     print(continentcode)
    ethnicity_continent = {'EU':'White/Caucasian', 'SA':'Hispanic/Latino', 'AS':'Asian', 'AF':'Black/African American', 'NA':'White/Caucasian', 'AN':'Other/Multiracial', 'OC':'Other/Multiracial','UNKNOWN':'UNKNOWN'}
    ethnicity_continent_notNA = {'EU':'White/Caucasian', 'SA':'Hispanic/Latino', 'AS':'Asian', 'AF':'Black/African American', 'NA':'Other/Multiracial', 'AN':'Other/Multiracial', 'OC':'Other/Multiracial','UNKNOWN':'UNKNOWN'}
    return ethnicity_continent_notNA[continentcode]
languageCode2ethnicity('pt')

'Hispanic/Latino'

# Read original data

In [4]:
candidate_data_list = list()
lang_data_list = list()
join_data_list = list()
original_dir = 'Result/original_data'
lang_result_dir = 'Result/language_result'
original_path = os.path.join(os.getcwd(),original_dir)
lang_result_path = os.path.join(os.getcwd(),lang_result_dir)
for root, dirs, files in os.walk(original_path):
#     print(dirs)
    for name in dirs:
        #load original file
        candidate_sub_list = list()
        original_subfolder_path = os.path.join(original_path,name)
        for i in range(0,10):
            filename = 'sample_data_1000_' + str(i)
            candidate_file = filename + '.csv'
            candidate_file_path = os.path.join(original_subfolder_path,candidate_file)
            candidate_df = pd.read_csv(candidate_file_path)
            candidate_sub_list.append(candidate_df)
            
        candidate_sub_df =  pd.concat(candidate_sub_list)
        candidate_sub_df = candidate_sub_df[['candidate_name','candidate_surname','candidate_country','ethnicity','ID','url',"cand_id"]]
        candidate_sub_df = candidate_sub_df.rename(columns={"cand_id": "candidate_id"})
        candidate_sub_df = candidate_sub_df.rename(columns={"ID": "cand_id"})
        print('length of original data %d'%len(candidate_sub_df))
        #load langresult file
        lang_sub_list = list()
        lang_subfolder_path = os.path.join(lang_result_path,name)
        for i in range(0,10):
            filename = 'sample_data_1000_%s_language'% str(i)
            lang_file = filename + '.csv'
            lang_file_path = os.path.join(lang_subfolder_path,lang_file)
            lang_df = pd.read_csv(lang_file_path)
            lang_sub_list.append(lang_df)
            
        lang_sub_df =  pd.concat(lang_sub_list)
        lang_sub_df = lang_sub_df.loc[lang_sub_df.proficiency == 'NATIVE_OR_BILINGUAL']
        lang_sub_df['language_code'] = lang_sub_df.name.apply(lambda row:nativeLang2LangCode(row))
        lang_sub_df['Predict_eth'] = lang_sub_df.language_code.apply(lambda row:languageCode2ethnicity(row))
        print('length of lang result data %d'%len(lang_sub_df))

        #Join data
        join_df = pd.merge(lang_sub_df,candidate_sub_df, how= 'left',on="cand_id" )
        join_df = join_df[join_df.ethnicity!='Other/Multiracial']
        join_df.to_csv('join_df_%s.csv'%name)
        join_data_list.append(join_df)
join_data_df = pd.concat(join_data_list)
join_data_df.shape

length of original data 1000
length of lang result data 330
length of original data 1000
length of lang result data 333


(576, 12)

# load modify file

In [5]:
join_data_df =pd.read_csv('join_data_df_newModify.csv')
len(join_data_df)

389

In [6]:
group = join_data_df.groupby('ethnicity').count()
group

,cand_id,name,proficiency,language_code,Predict_eth,candidate_name,candidate_surname,candidate_country,url,candidate_id
ethnicity,,,,,,,,,,
Asian,105,105,105,105,105,105,105,105,105,105
Black/African American,54,54,54,54,54,54,54,54,54,54
Hispanic/Latino,147,147,147,147,147,147,147,146,147,147
White/Caucasian,83,83,83,83,83,83,83,83,83,83


In [7]:
float(389/2000)

0.1945

In [8]:
# join_data_df.drop_duplicates(subset=['candidate_id','language_code'], keep=False)
join_data_df.head(5)

,cand_id,name,proficiency,language_code,Predict_eth,candidate_name,candidate_surname,candidate_country,ethnicity,url,candidate_id
0,10,English,NATIVE_OR_BILINGUAL,en,White/Caucasian,Saurabh,Singhal,US,Asian,https://www.linkedin.com/in/saurabh-singhal-b8...,125201
1,11,Vietnamese,NATIVE_OR_BILINGUAL,vi,Asian,Nguyen,Le,US,Asian,https://www.linkedin.com/in/nguyenle89,159427
2,13,Malayalam,NATIVE_OR_BILINGUAL,ml,Asian,Shamjith,antholi,CA,Asian,https://www.linkedin.com/in/shamjith-antholi-a...,7366
3,15,English,NATIVE_OR_BILINGUAL,en,White/Caucasian,Philippe,Thanadabouth,FR,Asian,https://www.linkedin.com/in/philippethanadabou...,15612
4,16,Chinese,NATIVE_OR_BILINGUAL,zh,Asian,Dong,Jiang,US,Asian,https://www.linkedin.com/in/djstudio?authType=...,122889


In [9]:
join_data_df.to_csv('join_data_newdf.csv')

# Analyzing result

Remove unknown result

In [10]:
join_data_df_notNAN = join_data_df.loc[join_data_df['Predict_eth']!='UNKNOWN']
len(join_data_df_notNAN)

355

### Raw result - include english, france

In [11]:
print(metrics.classification_report(join_data_df_notNAN['ethnicity'],join_data_df_notNAN['Predict_eth']))

                        precision    recall  f1-score   support

                 Asian       0.81      0.83      0.82        94
Black/African American       0.62      0.26      0.37        50
       Hispanic/Latino       0.90      0.69      0.78       133
     Other/Multiracial       0.00      0.00      0.00         0
       White/Caucasian       0.47      0.82      0.60        78

           avg / total       0.74      0.70      0.69       355



/Users/vunguyen/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [12]:
df_confusion = pd.crosstab(join_data_df_notNAN['ethnicity'],join_data_df_notNAN['Predict_eth'], rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

Predicted,Asian,Black/African American,Hispanic/Latino,Other/Multiracial,White/Caucasian,All
Actual,,,,,,
Asian,78,1,1,0,14,94
Black/African American,10,13,3,0,24,50
Hispanic/Latino,5,3,92,0,33,133
White/Caucasian,3,4,6,1,64,78
All,96,21,102,1,135,355


### Raw result - include france

In [13]:
join_data_df_notEng = join_data_df_notNAN.loc[join_data_df_notNAN['language_code']!='en']
len(join_data_df_notEng)

265

In [14]:
print(metrics.classification_report(join_data_df_notEng['ethnicity'],join_data_df_notEng['Predict_eth']))

                        precision    recall  f1-score   support

                 Asian       0.81      0.96      0.88        81
Black/African American       0.62      0.48      0.54        27
       Hispanic/Latino       0.90      0.81      0.86       113
     Other/Multiracial       0.00      0.00      0.00         0
       White/Caucasian       0.67      0.68      0.67        44

           avg / total       0.81      0.80      0.80       265



/Users/vunguyen/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [15]:
df_confusion_notEng = pd.crosstab(join_data_df_notEng['ethnicity'],join_data_df_notEng['Predict_eth'], rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion_notEng

Predicted,Asian,Black/African American,Hispanic/Latino,Other/Multiracial,White/Caucasian,All
Actual,,,,,,
Asian,78,1,1,0,1,81
Black/African American,10,13,3,0,1,27
Hispanic/Latino,5,3,92,0,13,113
White/Caucasian,3,4,6,1,30,44
All,96,21,102,1,45,265


### Raw result - remove english and france

In [16]:
join_data_df_notEngFrn = join_data_df_notEng.loc[join_data_df_notEng['language_code']!='fr']

In [17]:
print(metrics.classification_report(join_data_df_notEngFrn['ethnicity'],join_data_df_notEngFrn['Predict_eth']))

                        precision    recall  f1-score   support

                 Asian       0.81      0.97      0.89        80
Black/African American       0.92      0.44      0.59        25
       Hispanic/Latino       0.90      0.84      0.87       110
     Other/Multiracial       0.00      0.00      0.00         0
       White/Caucasian       0.67      0.73      0.70        41

           avg / total       0.84      0.82      0.82       256



/Users/vunguyen/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [18]:
df_confusion_notEng = pd.crosstab(join_data_df_notEngFrn['ethnicity'],join_data_df_notEngFrn['Predict_eth'], rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion_notEng

Predicted,Asian,Black/African American,Hispanic/Latino,Other/Multiracial,White/Caucasian,All
Actual,,,,,,
Asian,78,0,1,0,1,80
Black/African American,10,11,3,0,1,25
Hispanic/Latino,5,0,92,0,13,110
White/Caucasian,3,1,6,1,30,41
All,96,12,102,1,45,256


In [19]:
precision = np.mean(metrics.precision_recall_fscore_support(join_df['ethnicity'], join_df['Predict_eth'])[0])
recall = np.mean(metrics.precision_recall_fscore_support(join_df['ethnicity'], join_df['Predict_eth'])[1])
fscore = np.mean(metrics.precision_recall_fscore_support(join_df['ethnicity'], join_df['Predict_eth'])[2])
print(precision,recall,fscore)
metrics.classification_report

/Users/vunguyen/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.425201171185 0.328980405197 0.32625200568


<function sklearn.metrics.classification.classification_report>

In [20]:
df_confusion = pd.crosstab(join_df['ethnicity'], join_df['Predict_eth'], rownames=['Actual'], colnames=['Predicted'], margins=True)

df_confusion

Predicted,Asian,Black/African American,Hispanic/Latino,Other/Multiracial,UNKNOWN,White/Caucasian,All
Actual,,,,,,,
Asian,55,0,1,0,15,25,96
Black/African American,7,5,3,0,6,19,40
Hispanic/Latino,5,3,56,0,17,50,131
White/Caucasian,4,2,2,1,1,56,66
All,71,10,62,1,39,150,333


In [21]:
join_data_df_notEngFrArb = join_data_df_notEng.loc[join_data_df_notEng['language_code']!='ar']

In [22]:
print(metrics.classification_report(join_data_df_notEngFrArb['ethnicity'],join_data_df_notEngFrArb['Predict_eth']))

                        precision    recall  f1-score   support

                 Asian       0.81      0.96      0.88        81
Black/African American       0.63      0.46      0.53        26
       Hispanic/Latino       0.90      0.81      0.86       113
     Other/Multiracial       0.00      0.00      0.00         0
       White/Caucasian       0.67      0.70      0.68        43

           avg / total       0.81      0.81      0.80       263



/Users/vunguyen/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [23]:
df_confusion_notEng = pd.crosstab(join_data_df_notEngFrArb['ethnicity'],join_data_df_notEngFrArb['Predict_eth'], rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion_notEng

Predicted,Asian,Black/African American,Hispanic/Latino,Other/Multiracial,White/Caucasian,All
Actual,,,,,,
Asian,78,1,1,0,1,81
Black/African American,10,12,3,0,1,26
Hispanic/Latino,5,3,92,0,13,113
White/Caucasian,3,3,6,1,30,43
All,96,19,102,1,45,263
